# Nesting in the IPDL Model

In this notebook we introduce the IPDL nesting structure in a general setting. For this purpose we implement a concrete nesting structure using publically available data on the European car market from Frank Verboven's website at https://sites.google.com/site/frankverbo/data-and-software/data-set-on-the-european-car-market.

In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = "warn"
pd.set_option('display.max_rows', 500)
import os
import sys
from numpy import linalg as la
from scipy import optimize
import scipy.stats as scstat
from matplotlib import pyplot as plt
import itertools as iter
%load_ext line_profiler

# Files
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utilities.Logit_file import estimate_logit, logit_se, logit_t_p, q_logit, logit_score, logit_score_unweighted, logit_ccp, LogitBLP_estimator
from data.Eurocarsdata_file import Eurocars_cleandata

   variable names                                        description
0              cy            cylinder volume or displacement (in cc)
1              hp                                 horsepower (in kW)
2              we                                     weight (in kg)
3              le                                     length (in cm)
4              wi                                      width (in cm)
5              he                                     height (in cm)
6              li          average of li1, li2, li3 (used in papers)
7              sp                            maximum speed (km/hour)
8              ac  time to acceleration (in seconds from 0 to 100...
9              pr   price (in destination currency including V.A.T.)
10          brand                                      name of brand
11           home  domestic car dummy (appropriate interaction of...
12            cla                              class or segment code


x has full rank


In [ ]:
# Load dataset and variable names
# os.chdir('../GREENCAR_notebooks/') # Assigns work directory

input_path = os.getcwd() # Assigns input path as current working directory (cwd)
descr = (pd.read_stata('../data/eurocars.dta', iterator = True)).variable_labels() # Obtain variable descriptions
dat_file = pd.read_csv('../data/eurocars.csv') # reads in the data set as a pandas dataframe.

In [ ]:
pd.DataFrame(descr, index=['description']).transpose().reset_index().rename(columns={'index' : 'variable names'}) # Prints data sets

,variable names,description
0,ye,year (=first dimension of panel)
1,ma,market (=second dimension of panel)
2,co,model code (=third dimension of panel)
3,zcode,alternative model code (predecessors and succe...
4,brd,brand code
5,type,name of brand and model
6,brand,name of brand
7,model,name of model
8,org,"origin code (demand side, country with which c..."
9,loc,"location code (production side, country where ..."


In [ ]:
# Outside option is included if OO == True, otherwise analysis is done on the inside options only.
OO = True

# Choose which variables to include in the analysis, and assign them either as discrete variables or continuous.

x_discretevars = [ 'brand', 'home', 'cla']
x_contvars = ['cy', 'hp', 'we', 'le', 'wi', 'he', 'li', 'sp', 'ac', 'pr']
z_IV_contvars = ['xexr']
z_IV_discretevars = []
x_allvars =  [*x_contvars, *x_discretevars]
z_allvars = [*z_IV_contvars, *z_IV_discretevars]

if OO:
    nest_vars = [var for var in ['in_out', *x_allvars] if (var != 'pr')] # We nest over all variables other than price, but an alternative list can be specified here if desired.
else:
    nest_vars = [var for var in x_allvars if (var != 'pr')] # See above

nest_cont_vars = ['cy', 'hp', 'we', 'le', 'wi', 'he', 'li', 'sp', 'ac'] # The list of continuous variables, from which nests will be created according to the deciles of the distribution.

G = len(nest_vars)

# Print list of chosen variables as a dataframe
pd.DataFrame(descr, index=['description'])[x_allvars].transpose().reset_index().rename(columns={'index' : 'variable names'})

,variable names,description
0,cy,cylinder volume or displacement (in cc)
1,hp,horsepower (in kW)
2,we,weight (in kg)
3,le,length (in cm)
4,wi,width (in cm)
5,he,height (in cm)
6,li,"average of li1, li2, li3 (used in papers)"
7,sp,maximum speed (km/hour)
8,ac,time to acceleration (in seconds from 0 to 100...
9,pr,price (in destination currency including V.A.T.)


In [ ]:
dat, dat_org, x_vars, z_vars, N, pop_share, T, J, K = Eurocars_cleandata(dat_file, x_contvars, x_discretevars, z_IV_contvars, z_IV_discretevars, outside_option=OO)

In [ ]:
# Create dictionaries of numpy arrays for each market. This allows the size of the data set to vary over markets.

dat = dat.reset_index(drop = True).sort_values(by = ['market', 'co']) # Sort data so that reshape is successfull

x = {t: dat[dat['market'] == t][x_vars].values.reshape((J[t],K)) for t in np.arange(T)} # Dict of explanatory variables
y = {t: dat[dat['market'] == t]['ms'].to_numpy().reshape((J[t])) for t in np.arange(T)} # Dict of market shares

### Nesting in the IPDL Model
We will start our discussion by considering a related model: the Nested Logit Model. In the Nested Logit Model, the choice set $\mathcal{J}_t = \{1,\ldots,J_t\}$ is divided into a partition $\mathcal{C} = \{C_1, \ldots, C_L\}$, such that each alternative $j$ is in exactly one of the nests $C_\ell$. Given such a partition, we construct a matrix $\psi \in \mathbb{R}^{L\times J_t}$ representing whether alternative $j$ is in some nest $C_\ell$, by setting $\psi_{\ell j} = 1$ if alternative $j$ is in nest $\ell$, and $ \psi_{\ell j} = 0$ else.

The IPDL Model generalizes this structure, by considering the case of including multiple partions $\mathcal{C}_1, \ldots, \mathcal{C}_G$ of the choice set at the same time. In analogy to the Nested Logit Model, we may construct so called 'incidence matrices' $\psi^1, \ldots, \psi^G$, each of which similarly indicates whether or not an alternative $j$ belongs to the $\ell$'th nest $C_\ell^g$ of some partition $\mathcal{C}_g$, $g = 1, \ldots, G$. In practice, one may for example wish to nest alternative according to a subset of the explanatory variables. In the following, we define a function which constructs nests directly from any inputted dataset.

In [ ]:
def Create_nests(data, markets_id, products_id, columns, cont_var = None, cont_var_bins = None, outside_option = True):
    '''
    This function creates the nest matrices \Psi^{gt}, and stack them over g for each t.

    Args.
        data: a pandas DataFrame
        markets_id: a string denoting the column of 'data' containing an enumeration t=0,1,...,T-1 of markets
        products_id: a string denoting the column of 'data' containing product codes which uniquely identifies products
        columns: a list containing the column names of columns in 'data' from which nest groupings g=0,1,...,G-1 for each market t are to be generated
        cont_var: a list of the continuous variables in 'columns'
        cont_var_bins: a list containing the number of bins to make for each continuous variable in 'columns'
        outside_option: a boolean indicating whether the model is estimated with or without an outside option. Default is set to 'True' i.e. with an outside option.

    Returns
        Psi: a dictionary of length T of the J[t] by J[t] identity stacked on top of the Psi_g matrices for each market t and each gropuing g
        nest_dict: a dictionary of length T of pandas series describing the structure of each nest for each market t and each grouping g
        nest_count: a dictionary of length T of (G,) numpy arrays containing the amount of nests in each category g
    '''

    T = data[markets_id].nunique()
    J = np.array([data[data[markets_id] == t][products_id].nunique() for t in np.arange(T)])
    
    # We include nest on outside vs. inside options. The amount of categories varies if the outside option is included in the analysis.
    dat = data.sort_values(by = [markets_id, products_id]) # We sort the data in ascending, first according to market and then according to the product id
    
    Psi = {}
    nest_dict = {}
    nest_counts = {}

    # Assign nests for products in each market t
    for t in np.arange(T):
        data_t = dat[dat[markets_id] == t] # Subset data on market t
        nest_frame = data_t.copy()

        ### Bin continuous variables according to quantiles of the variable

        if cont_var == None:
            None
        else:
            for var,n_bins in zip(cont_var,cont_var_bins):
                if outside_option:
                    q_dat = np.unique(np.quantile(data_t[var].rank(method = 'min'), q = np.arange(1,n_bins + 1) / n_bins)) # Get the unique 'n_bins' equally spaced quantiles of each continuous variable given in the cont_var list
                    nest_frame[var] = pd.cut(data_t[var].rank(method = 'min'), bins = [0.99,1, *q_dat], labels=False) # Quantiles are equally spaced with 'n_bins' quantiles for the variable. The outside option gets its own bin (0.99,1].
                else:
                    q_dat = np.unique(np.quantile(data_t[var].rank(method = 'min'), q = np.arange(1,n_bins + 1) / n_bins)) # Get the unique 'n_bins' equally spaced quantiles of each continuous variable given in the cont_var list
                    nest_frame[var] = pd.cut(data_t[var].rank(method = 'min'), bins = q_dat, labels=False) # Bin the variable according to 'n_bins' equally spaced quantiles.

        nest_dict[t] = nest_frame[columns].apply(lambda col: list(np.unique(col))) # Get the unique values of each 'col' in columns
        nest_counts[t] = nest_frame[columns].nunique().values # Find the number of unique values in each column in columns and output as a numpy array

        nest_count_total = nest_frame[columns].nunique().sum() # Find the sum of nest counts L_g
        nests = pd.get_dummies(nest_frame[columns], columns = columns).values.reshape((J[t], nest_count_total)).transpose() # Finds dummies for each category in columns, and converts these to numpy arrays of the appropiate size. Note that the data has been sorted according to market and then product.
        Psi_t = np.concatenate([np.eye(J[t]), nests], axis = 0) # Stack a J[t] by J[t] identity on top of the stacked \Psi^g matrices for each market t

        Psi[t] = Psi_t

    return Psi, nest_dict, nest_counts

In [ ]:
cont_bins=[np.int64(10) for i in range(len(nest_cont_vars))] # Sets the number of bins to 10 for each continuous variable.
Psi, Nest_descr, Nest_count = Create_nests(dat, 'market', 'co', nest_vars, nest_cont_vars, cont_bins , outside_option=OO)